# Object Detection 

In [39]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.rpn import AnchorGenerator
import numpy
import matplotlib.pyplot
import cv2
import json
import time

import utils
from coco_utils import get_coco_api_from_dataset
from coco_eval import CocoEvaluator

%matplotlib inline

## Import Dataset 

In [7]:
cap = torchvision.datasets.CocoDetection(root = 'data/val2017',
                                        annFile = 'data/annotations/stuff_val2017.json',
                                        transform=torchvision.transforms.ToTensor())

test_sampler = torch.utils.data.SequentialSampler(cap)

loading annotations into memory...
Done (t=0.38s)
creating index...
index created!


In [31]:
data_loader = torch.utils.data.DataLoader(cap,
                                          batch_size=1,
                                          shuffle=True,
                                          num_workers=0)

In [32]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

In [41]:
def _get_iou_types(model):
    model_without_ddp = model
    if isinstance(model, torch.nn.parallel.DistributedDataParallel):
        model_without_ddp = model.module
    iou_types = ["bbox"]
    if isinstance(model_without_ddp, torchvision.models.detection.MaskRCNN):
        iou_types.append("segm")
    if isinstance(model_without_ddp, torchvision.models.detection.KeypointRCNN):
        iou_types.append("keypoints")
    return iou_types

@torch.no_grad()
def evaluate(model, data_loader):
    n_threads = torch.get_num_threads()
    # FIXME remove this and make paste_masks_in_image run on the GPU
    torch.set_num_threads(1)
    cpu_device = torch.device("cpu")
    model.eval()
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = 'Test:'

    coco = get_coco_api_from_dataset(data_loader.dataset)
    iou_types = _get_iou_types(model)
    coco_evaluator = CocoEvaluator(coco, iou_types)
    print('[INFO] before batch')

    for images, targets in metric_logger.log_every(data_loader, 100, header):
        images = list(img.to(cpu_device) for img in images)
        targets = [{k: v for k, v in t.items()} for t in targets]

        model_time = time.time()
        outputs = model(images)

        outputs = [{k: v for k, v in t.items()} for t in outputs]
        model_time = time.time() - model_time

        res = {target["image_id"].item(): output for target, output in zip(targets, outputs)}
        evaluator_time = time.time()
        coco_evaluator.update(res)
        evaluator_time = time.time() - evaluator_time
        metric_logger.update(model_time=model_time, evaluator_time=evaluator_time)
        print('[INFO] batch')

    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    coco_evaluator.synchronize_between_processes()

    # accumulate predictions from all images
    coco_evaluator.accumulate()
    coco_evaluator.summarize()
    torch.set_num_threads(n_threads)
    return coco_evaluator

In [42]:
evaluate(model, data_loader)

[INFO] before batch


ImportError: /home/ava/anaconda3/lib/python3.7/site-packages/torchvision/_C.so: undefined symbol: _ZN6caffe26detail37_typeMetaDataInstance_preallocated_29E